## Multivariate LSTM-FCN
- Code source : https://github.com/metra4ok/MLSTM-FCN-Pytorch
- Modified some sturcture and parameters w.r.t. our IOT hackathon data

In [2]:
from google.colab import drive
drive.mount('/content/Mydrive')
%cd "/content/Mydrive/MyDrive/IOT_hackathon/Github/IOT_hackathon"
%cd "/content/Mydrive/MyDrive/IOT_hackathon/Github/IOT_hackathon/LSTM-FCN"

Mounted at /content/Mydrive
/content/Mydrive/MyDrive/IOT_hackathon/Github/IOT_hackathon
/content/Mydrive/MyDrive/IOT_hackathon/Github/IOT_hackathon/LSTM-FCN


In [3]:
from src.model import SELayer, MLSTMfcn
from src.dataset import load_datasets
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F 
import torch.optim as optim
import random
import os
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

In [ ]:
# Seed and Device setting

def def_seed(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

def_seed(seed=123)

if torch.has_cuda:
    device = torch.device('cuda') 
elif torch.has_mps:
    device = torch.device('mps')
else:
    device = torch.device('cpu')
print(device) # Print Using Device

mps


### Tensor Structure
- shape : [NUM_DATA, MAX_LENGTH, NUM_FEATURE]
> 각 데이터별로(grid) time step에서의 state(value of each feature)을 1차원 텐서로, 모든 time step
> 핵심 : time step의 최대길이는 맞춤(존재하지 않는다면, 0으로 값을 주어 길이를 동일하게)

In [ ]:
def validation(model, testloader, criterion, device=device):
    accuracy = 0
    test_loss = 0
    for inputs, labels, seq_lens in testloader:
        inputs = inputs.float()
        inputs, labels = inputs.to(device), labels.to(device)

        output = model.forward(inputs, seq_lens)
        test_loss += criterion(output, labels).item()

        ## Calculating the accuracy 
        # Model's output is log-softmax, take exponential to get the probabilities
        ps = torch.exp(output)
        # Class with highest probability is our predicted class, compare with true label
        equality = (labels.data == ps.max(1)[1])
        # Accuracy is number of correct predictions divided by all predictions, just take the mean
        accuracy += equality.type_as(torch.FloatTensor()).mean()

    return test_loss, accuracy


def train(model, trainloader, validloader, criterion, optimizer, 
          epochs=10, print_every=10, device=device, run_name='model_mlstm_fcn'):
    print("Training started on device: {}".format(device))

    valid_loss_min = np.Inf # track change in validation loss
    steps = 0
    
    for e in range(epochs):
        train_loss = 0.0
        valid_loss = 0.0

        model.train()
        for inputs, labels, seq_lens in trainloader:
            steps += 1

            inputs = inputs.float()
            inputs, labels = inputs.to(device),labels.to(device)
            
            optimizer.zero_grad()
            
            output = model.forward(inputs, seq_lens)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()

            if steps % print_every == 0:
                model.eval()
                
                with torch.no_grad():
                    valid_loss, accuracy = validation(model, validloader, criterion, device)
                
                print("Epoch: {}/{}.. ".format(e+1, epochs),
                      "Training Loss: {:.6f}.. ".format(train_loss/print_every),
                      "Val Loss: {:.6f}.. ".format(valid_loss/len(validloader)),
                      "Val Accuracy: {:.2f}%".format(accuracy/len(validloader)*100))
                
                # save model if validation loss has decreased
                if valid_loss <= valid_loss_min:
                    print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
                    valid_loss_min,
                    valid_loss))
                    torch.save(model.state_dict(), 'weights/'+run_name+'.pt')
                    valid_loss_min = valid_loss

                train_loss = 0

                model.train()

False

In [ ]:
# Hyperparameters
epochs = 100
lr = 0.01
bs = 1024
# variable
NUM_CLASSES = 2
MAX_SEQ_LEN = 8640 # 24*365
NUM_FEATURES = 28

## Model Training

In [ ]:
train_dataset, val_dataset, _ = load_datasets()

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = bs)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size = bs)

In [ ]:
# Train
mlstm_fcn_model = MLSTMfcn(
    num_classes=NUM_CLASSES,
    max_seq_len=MAX_SEQ_LEN,
    num_features=NUM_FEATURES
)
mlstm_fcn_model.to(device)

optimizer = optim.Adam(mlstm_fcn_model.parameters(), lr=lr)
criterion = nn.NLLLoss()

train(mlstm_fcn_model, train_loader, val_loader, criterion, optimizer, epochs, print_every=100, device=device, run_name="model_mlstm_fcn")

2

In [ ]:
# Save model
torch.save(mlstm_fcn_model.state_dict())